# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [14]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [15]:
weather_df = pd.read_csv('Resources/Cities.csv')
weather_df = weather_df.drop(columns=['Unnamed: 0'])
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Faanui,-16.48,-151.75,80.40,77,68,17.83,PF,1603752984
1,Antofagasta,-23.65,-70.40,60.80,72,90,13.87,CL,1603753336
2,Øksfjord,70.24,22.35,35.60,73,0,13.87,NO,1603753473
3,Jamestown,42.10,-79.24,45.00,93,90,4.70,US,1603752995
4,Havre-St-Pierre,50.23,-63.60,32.00,80,90,4.70,CA,1603753210
...,...,...,...,...,...,...,...,...,...
579,Rønne,55.10,14.71,51.80,100,100,8.05,DK,1603753543
580,Urubichá,-15.38,-62.95,70.75,97,100,2.93,BO,1603753543
581,Yulara,-25.24,130.99,64.40,27,0,10.29,AU,1603753169
582,Tongzhou,39.91,116.60,57.00,55,0,4.47,CN,1603753544


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
gmaps.configure(api_key=g_key)

In [17]:
fig = gmaps.figure()

locations = weather_df[['Lat', 'Lng']]
weights = weather_df['Humidity']
figure_layout = {'width': '500px', 'margin': '0 auto 0 auto'}
gmaps.figure(layout=figure_layout)
gmaps.heatmap_layer.max_intensity = 200
gmaps.heatmap_layer.point_radius = 5
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
# selecting rows based on condition 
hotel_df = weather_df.loc[(weather_df['Max Temp'] > 65) & (weather_df['Max Temp'] < 70) & 
                         (weather_df['Wind Speed'] < 15) & 
                         (weather_df['Cloudiness'] < 10)]
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Jalu,29.03,21.55,67.64,56,2,4.23,LY,1603753479
1,Abu Samrah,35.30,37.18,69.12,23,0,11.50,SY,1603753482
2,Bhadrāchalam,17.67,80.88,68.72,85,0,2.57,IN,1603753483
3,Gohpur,26.88,93.63,69.58,89,1,1.90,IN,1603753489
4,Bilma,18.69,12.92,68.70,24,0,1.32,NE,1603753497
5,Alice Springs,-23.70,133.88,66.20,24,0,10.29,AU,1603753139
6,Myitkyina,25.38,97.40,69.31,86,0,1.68,MM,1603753516
7,Tura,25.52,90.22,68.76,92,0,2.93,IN,1603753062
8,Biswān,27.50,81.00,66.67,32,0,5.55,IN,1603753520
9,Presidencia Roque Sáenz Peña,-26.79,-60.44,67.35,62,0,5.66,AR,1603753522


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [19]:
hotel_df['Nearby Hotel'] = ''
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Nearby Hotel
0,Jalu,29.03,21.55,67.64,56,2,4.23,LY,1603753479,
1,Abu Samrah,35.30,37.18,69.12,23,0,11.50,SY,1603753482,
2,Bhadrāchalam,17.67,80.88,68.72,85,0,2.57,IN,1603753483,
3,Gohpur,26.88,93.63,69.58,89,1,1.90,IN,1603753489,
4,Bilma,18.69,12.92,68.70,24,0,1.32,NE,1603753497,
5,Alice Springs,-23.70,133.88,66.20,24,0,10.29,AU,1603753139,
6,Myitkyina,25.38,97.40,69.31,86,0,1.68,MM,1603753516,
7,Tura,25.52,90.22,68.76,92,0,2.93,IN,1603753062,
8,Biswān,27.50,81.00,66.67,32,0,5.55,IN,1603753520,
9,Presidencia Roque Sáenz Peña,-26.79,-60.44,67.35,62,0,5.66,AR,1603753522,


In [20]:
# hotels = []

# for row in range(len(hotel_df)):
#     lat = hotel_df.loc[row]['Lat']
#     lng = hotel_df.loc[row]['Lng']
    
#     params = {
#         'location' = (f'{lat},{lng}'),
#         'radius' = 5000,
#         'types' : 'hotel',
#         'key' = g_key
#     }
    
#     qry_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
#     response = requests.get(base_url, params = params).json()
    
#     try:
#         hotel.append(json['results'][0]['name'])
#     except:
#         hotel.append("Not Found")

# hotel_df['Nearby Hotel'] = hotels

qry_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key
         }

In [21]:
# Read through rows and grab info needed to preform search

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"

    # URL and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(qry_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 0: Jalu.
Closest hotel in Jalu is Jalu Hotel.
------------
Retrieving Results for Index 1: Abu Samrah.
Missing field/result... skipping.
------------
Retrieving Results for Index 2: Bhadrāchalam.
Closest hotel in Bhadrāchalam is Hotel Nandini.
------------
Retrieving Results for Index 3: Gohpur.
Missing field/result... skipping.
------------
Retrieving Results for Index 4: Bilma.
Missing field/result... skipping.
------------
Retrieving Results for Index 5: Alice Springs.
Closest hotel in Alice Springs is DoubleTree by Hilton Hotel Alice Springs.
------------
Retrieving Results for Index 6: Myitkyina.
Closest hotel in Myitkyina is Palm Spring Resort Hotel Myitkyina.
------------
Retrieving Results for Index 7: Tura.
Closest hotel in Tura is Hotel Polo Orchid.
------------
Retrieving Results for Index 8: Biswān.
Closest hotel in Biswān is Siraj Hotal.
------------
Retrieving Results for Index 9: Presidencia Roque Sáenz Peña.
Closest hotel in Presidencia Roqu

In [22]:
hotel_df = hotel_df.dropna()
hotel_df = hotel_df.reset_index(drop=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Nearby Hotel,Hotel Name
0,Jalu,29.03,21.55,67.64,56,2,4.23,LY,1603753479,,Jalu Hotel
1,Bhadrāchalam,17.67,80.88,68.72,85,0,2.57,IN,1603753483,,Hotel Nandini
2,Alice Springs,-23.70,133.88,66.20,24,0,10.29,AU,1603753139,,DoubleTree by Hilton Hotel Alice Springs
3,Myitkyina,25.38,97.40,69.31,86,0,1.68,MM,1603753516,,Palm Spring Resort Hotel Myitkyina
4,Tura,25.52,90.22,68.76,92,0,2.93,IN,1603753062,,Hotel Polo Orchid
5,Biswān,27.50,81.00,66.67,32,0,5.55,IN,1603753520,,Siraj Hotal
6,Presidencia Roque Sáenz Peña,-26.79,-60.44,67.35,62,0,5.66,AR,1603753522,,Hotel Gualok
7,Nokha,27.60,73.42,68.05,24,0,3.33,IN,1603753524,,Hotel Gaytri Palace
8,Caraí,-17.19,-41.69,66.31,95,2,0.89,BR,1603753526,,Pousada Graal De Carai
9,Los Llanos de Aridane,28.66,-17.92,69.80,64,0,10.29,ES,1603753528,,Hotel Hacienda de Abajo


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
# End